In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import os
import torch
import torch.nn as nn
import torch.onnx
import yaml
import onnx
import onnxruntime
import numpy as np
from PIL import Image
import cv2

from model import CRNNet, MORANet, VGG_UNet
from net import CRNN, MORAN, CRAFT
from tools.imgproc import resize_aspect_ratio, normalizeMeanVariance

In [3]:
with open(os.path.join('config.yml'),'r') as f:
    config = yaml.safe_load(f)
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model_path = os.path.join(config['pretrained'], 'CRNN.pth')
craft_onnx_path = os.path.join(config['onnx_path'], 'CRNN.onnx')
os.makedirs(config['onnx_path'], exist_ok=True)

In [4]:
batch_size=1
model =CRNNet(config).to(device)
model.load_state_dict(torch.load(model_path))

<All keys matched successfully>

In [18]:
model.eval()
text_pred = torch.LongTensor(batch_size, config['batch_max_len'] + 1).fill_(0).to(device)
x = torch.randn(batch_size, 1, 244,244, requires_grad=True).to(device)
torch_outs = model(x, text_pred, training=False)

In [19]:
torch.onnx.export(model, (x,text_pred), craft_onnx_path, export_params=True,
                  opset_version=11, do_constant_folding=True,input_names = ['inputs'],
                  output_names = ['outputs'],dynamic_axes={'inputs' : {0 : 'batch_size'},
                                                           'outputs' : {0 : 'batch_size'}})

/mnt/Centralized/documents/cs/lightly-ocr/ocr/modules/biLSTM.py:59: TracerWarning: Converting a tensor to a Python index might cause the trace to be incorrect. We can't record the data flow of Python values, so this value will be treated as a constant in the future. This means that the trace might not generalize to other inputs!
  
/mnt/Centralized/documents/cs/lightly-ocr/ocr/modules/biLSTM.py:60: TracerWarning: Converting a tensor to a Python index might cause the trace to be incorrect. We can't record the data flow of Python values, so this value will be treated as a constant in the future. This means that the trace might not generalize to other inputs!
  def forward(self, batch_hidden, text, training=True, batch_max_len=25):
/mnt/Centralized/documents/cs/lightly-ocr/ocr/modules/biLSTM.py:50: TracerWarning: Converting a tensor to a Python index might cause the trace to be incorrect. We can't record the data flow of Python values, so this value will be treated as a constant in the fu

RuntimeError: Exporting the operator grid_sampler to ONNX opset version 11 is not supported. Please open a bug to request ONNX export support for the missing operator.

### 

In [ ]:
craft_onnx = onnx.load(craft_onnx_path)
onnx.checker.check_model(craft_onnx)
# onnx.helper.printable_graph(craft_onnx.graph)

In [ ]:
sess = onnxruntime.InferenceSession(craft_onnx_path)
# sess.get_inputs()[0].shape

In [ ]:
def to_numpy(tensor):
    try:
        if tensor.requires_grad:
            return tensor.detach().cpu().numpy()
    except:
        return tensor.cpu().numpy()

In [ ]:
# ort_inputs = {sess.get_inputs()[0].name: to_numpy(x)}
# ort_outs = sess.run(None, ort_inputs)
# np.testing.assert_allclose(to_numpy(torch_outs[0]), ort_outs[0], rtol=1e-03, atol=1e-05)

In [ ]:
image = cv2.imread('test/test1.png')

In [ ]:
image.size

In [ ]:
img_resized, target_ratio, size_heatmap = resize_aspect_ratio(image,1280,
                                                              interpolation=cv2.INTER_LINEAR,
                                                              mag_ratio=1.5)
ratio_h = ratio_w = 1 / target_ratio
img = normalizeMeanVariance(img_resized)
img = torch.from_numpy(img).permute(2, 0, 1)  # [h x w x c] -> [c x h x w]
img = torch.Tensor(img.unsqueeze(0)).to(device)

In [ ]:
ort_inputs = {sess.get_inputs()[0].name: to_numpy(img)}

In [ ]:
ort_inputs

In [ ]:
ort_outs = sess.run(None, ort_inputs)
img_out = ort_outs[0]